In [1]:
import transformers

In [2]:
from transformers import BertTokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

In [64]:
import pickle
import pandas as pd
from tqdm.notebook import tqdm

In [65]:
file = "all_data_test.pk"
with open(file, "rb",) as f:
    obj = pickle.load(f)
df = pd.DataFrame(obj)

print(df.columns)
print(df.head())

Index(['id', 'actual_cui', 'sentence', 'start', 'end', 'definition', 'cui'], dtype='object')
   id actual_cui                                           sentence  start  \
0   0   C0019699  004668411 CTMC 68299235 763052 9/29/1993 12:00...    232   
1   0   C0019699  004668411 CTMC 68299235 763052 9/29/1993 12:00...    232   
2   0   C0019699  004668411 CTMC 68299235 763052 9/29/1993 12:00...    232   
3   0   C0019699  004668411 CTMC 68299235 763052 9/29/1993 12:00...    232   
4   0   C0019699  004668411 CTMC 68299235 763052 9/29/1993 12:00...    232   

   end                                         definition       cui  
0  244  HIV Seropositivity means development of neutra...  C0019699  
1  244  HIV Seropositivity means zvýšená hladina neutr...  C0019699  
2  244  HIV Seropositivity means infected with the hum...  C0019699  
3  244  HIV Seropositivity means a laboratory test res...  C0019699  
4  244  HIV Seropositivity means o desenvolvimento de ...  C0019699  


In [66]:
MAX_LEN = 256

In [67]:
# we fix that the input will be divided into 2 parts. 1 for sentence and 2nd for defination
tokens, attention, token_type_ids = [], [] ,[]
res = []
for index in tqdm(df.index):
    row = df.loc[index]
    d = {'id': row['id'], 'cui': row['cui'], 'actual_cui': row['actual_cui']}
    sentence, start, end, defination = row.sentence, row.start, row.end, row.definition
    
    token_pre = tokenizer.tokenize(sentence[:start])
    token_mention = tokenizer.tokenize(sentence[start:end])
    token_post = tokenizer.tokenize(sentence[end:])
    
    token_definition = tokenizer.tokenize(defination)
    
    if len(token_definition) + 1 >= MAX_LEN//2:
        token_definition = token_definition[:MAX_LEN//2-1] + ['[SEP]']
        available_len = MAX_LEN//2
    else:
        available_len = MAX_LEN - len(token_definition)
    
    if 1 + len(token_pre) + len(token_mention) + len(token_post) + 1 >= available_len:
        available_space = available_len - len(token_mention) - 1 - 1
        
        l_space = available_space//2 - len(token_pre)
        r_space = available_space//2 - len(token_post)
        
        if l_space < 0 and r_space < 0:
            token_pre = token_pre[-available_space//2:]
            token_post = token_post[:available_space//2]
        
        elif l_space < 0 and r_space>=0:
            token_pre = token_pre[-(available_space//2 + r_space):]
        
        elif l_space >=0 and r_space < 0:
            token_post = token_post[:(available_space//2 + l_space)]
        
        elif l_space >=0 and r_space >= 0:
            pass
    token_pre = ['[CLS]'] + token_pre
    token_post = token_post + ['[SEP]']

    token_start = len(token_pre)
    token_end = token_start + len(token_mention)
    
#     if len(token_pre) + len(token_mention) + len(token_post) + len(token_definition) != MAX_LEN:
#         print(len(token_pre) , len(token_mention) , len(token_post) , len(token_definition), 
#              len(token_pre) + len(token_mention) + len(token_post) + len(token_definition))
    
    ids = tokenizer.convert_tokens_to_ids(token_pre + token_mention + token_post + token_definition)
    input_type_ids = [0 for _ in token_pre] + [0 for _ in token_mention] + [0 for _ in token_post] + [1 for _ in token_definition]
    attention = [1 for _ in range(len(ids))] + [0 for _ in range(MAX_LEN - len(ids))]
    
    if len(ids) < MAX_LEN:
        ids.extend([101 for _ in range(MAX_LEN-len(ids))])
        input_type_ids.extend([1 for _ in range(MAX_LEN-len(input_type_ids))])
        attention.extend([0 for _ in range(MAX_LEN-len(attention))])
    assert len(ids) == MAX_LEN
    assert len(input_type_ids) == MAX_LEN
    assert len(attention) == MAX_LEN
    d['input_ids'] = ids
    d['token_type_ids'] = input_type_ids
    d['attention_mask'] = attention
    
    res.append(d)

In [68]:
# dataframe 
d = pd.DataFrame(res)

In [69]:
d.shape

(147761, 6)

In [47]:
d['actual_cui'] = df2['actual_cui']
d.head()

,id,cui,input_ids,token_type_ids,attention_mask,actual_cui
0,1,C0006138,"[101, 11414, 1753, 138, 8661, 9538, 2165, 160,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,1,C0006138,"[101, 141, 120, 140, 23066, 20427, 2069, 139, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
2,1,C0006138,"[101, 9919, 1335, 22835, 131, 145, 12420, 2137...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
3,1,C0030193,"[101, 120, 4991, 13020, 1367, 131, 3135, 14123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
4,1,C0030193,"[101, 120, 1492, 120, 4991, 13020, 1367, 131, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193


In [70]:
d.to_pickle("BertNSPInput_test.pk")

In [56]:
d_ = d.set_index(['id'])

In [63]:
temp = d_.xs(1)
temp[temp['cui'] == temp['actual_cui']]

,cui,input_ids,token_type_ids,attention_mask,actual_cui
id,,,,,
1,C0030193,"[101, 120, 4991, 13020, 1367, 131, 3135, 14123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 120, 1492, 120, 4991, 13020, 1367, 131, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 1367, 120, 1492, 120, 4991, 13020, 1367,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 15681, 20427, 2162, 149, 119, 117, 150, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 17656, 2162, 141, 120, 140, 23066, 20427...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 15681, 20427, 2162, 149, 119, 117, 150, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 15740, 11680, 1942, 19753, 11414, 1753, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 1942, 21748, 131, 1367, 120, 1492, 120, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 14123, 18732, 15681, 15740, 11680, 1942,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193


In [58]:
d_

,cui,input_ids,token_type_ids,attention_mask,actual_cui
id,,,,,
1,C0006138,"[101, 11414, 1753, 138, 8661, 9538, 2165, 160,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0006138,"[101, 141, 120, 140, 23066, 20427, 2069, 139, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0006138,"[101, 9919, 1335, 22835, 131, 145, 12420, 2137...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 120, 4991, 13020, 1367, 131, 3135, 14123...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
1,C0030193,"[101, 120, 1492, 120, 4991, 13020, 1367, 131, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0030193
...,...,...,...,...,...
6697,C0085166,"[101, 119, 1247, 1108, 1185, 2455, 13585, 4267...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0232326
6697,C0085166,"[101, 153, 20996, 1116, 1127, 1155, 3253, 185,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0232326
6697,C0700329,"[101, 3848, 2116, 114, 119, 1109, 5351, 1108, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",C0232326
